# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
## nltk.download('averaged_perceptron_tagger')
## nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
# ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("pset3_inputdata/combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string
subset_pharma = doj[doj.id == "17-1204"]
pharma = "".join(subset_pharma["contents"].astype(str))
type(pharma)

str

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
## your code here to restrict to alpha
pharma_pr = [word for word in wordpunct_tokenize(pharma) 
            if word.isalpha()]

pharma_pr = " ".join(pharma_pr)

In [5]:
## your code here for part of speech tagging
tokens = word_tokenize(pharma_pr) # Generate list of tokens
tokens_pos = pos_tag(tokens) # generate part of speech tags for those tokens
tokens_pos

[('The', 'DT'),
 ('founder', 'NN'),
 ('and', 'CC'),
 ('majority', 'NN'),
 ('owner', 'NN'),
 ('of', 'IN'),
 ('Insys', 'NNP'),
 ('Therapeutics', 'NNP'),
 ('Inc', 'NNP'),
 ('was', 'VBD'),
 ('arrested', 'VBN'),
 ('today', 'NN'),
 ('and', 'CC'),
 ('charged', 'VBN'),
 ('with', 'IN'),
 ('leading', 'VBG'),
 ('a', 'DT'),
 ('nationwide', 'JJ'),
 ('conspiracy', 'NN'),
 ('to', 'TO'),
 ('profit', 'VB'),
 ('by', 'IN'),
 ('using', 'VBG'),
 ('bribes', 'NNS'),
 ('and', 'CC'),
 ('fraud', 'NN'),
 ('to', 'TO'),
 ('cause', 'VB'),
 ('the', 'DT'),
 ('illegal', 'JJ'),
 ('distribution', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('Fentanyl', 'NNP'),
 ('spray', 'NN'),
 ('intended', 'VBD'),
 ('for', 'IN'),
 ('cancer', 'NN'),
 ('patients', 'NNS'),
 ('experiencing', 'VBG'),
 ('breakthrough', 'NN'),
 ('pain', 'NN'),
 ('More', 'JJR'),
 ('than', 'IN'),
 ('Americans', 'NNPS'),
 ('died', 'VBD'),
 ('of', 'IN'),
 ('synthetic', 'JJ'),
 ('opioid', 'NN'),
 ('overdoses', 'NNS'),
 ('last', 'JJ'),
 ('year', 'NN'),
 ('and', 'CC'),
 (

In [6]:
## part C
adj = [token[0] for token in tokens_pos if token[1] == "JJ" or token[1] == "JJN" or token[1] == "JJS"]
adj_counts = {}
for a in adj:
    adj_counts[a] = adj_counts.get(a, 0) + 1

sorted_adj = sorted(adj_counts.items(), key=lambda x: x[1], reverse=True)

top5_adj = pd.DataFrame(sorted_adj[:5], columns=["Adjective", "Count"])
top5_adj

,Adjective,Count
0,former,8
1,opioid,5
2,nationwide,4
3,other,3
4,addictive,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [7]:
## your code here for part A
pharma_ner = nlp(pharma)
for tok in pharma_ner.ents:
    print(tok.text + "; " + tok.label_)

Insys Therapeutics Inc.; ORG
today; DATE
Fentanyl; PERSON
More than 20,000; CARDINAL
Americans; NORP
last year; DATE
millions; CARDINAL
Jeff Sessions; PERSON
This Justice Department; ORG
Trump; PERSON
American; NORP
”John N. Kapoor; PERSON
74; DATE
Phoenix; GPE
Ariz.; GPE
the Board of Directors; ORG
Insys; ORG
this morning; TIME
Arizona; GPE
RICO; LAW
Kapoor; PERSON
Executive; ORG
Board; ORG
Insys; ORG
Phoenix; GPE
today; DATE
U.S.; GPE
District Court; ORG
Boston; GPE
a later date; DATE
today; DATE
Boston; GPE
Insys; ORG
December 2016.The; DATE
Kapoor; GPE
Michael L. Babich; PERSON
40; DATE
Scottsdale; GPE
Ariz.; GPE
Alec Burlakoff; PERSON
42; DATE
Charlotte; GPE
N.C.; GPE
Richard M. Simon; PERSON
46; DATE
Seal Beach; GPE
Calif.; GPE
Sunrise Lee; PERSON
36; DATE
Bryant City; GPE
Mich.; GPE
Joseph A. Rowan; PERSON
43; DATE
Panama City; GPE
Fla.; GPE
Managed Markets; ORG
Michael J. Gurry; PERSON
53; DATE
Scottsdale; GPE
Ariz.; GPE
Subsys; ORG
Kapoor; GPE
six; CARDINAL
Kapoor; PERSON
Unit

In [8]:
## your code here for part B
law_ents = [ent.text for ent in pharma_ner.ents if ent.label_ == "LAW"]
print(law_ents)

['RICO', 'the Controlled Substances Act', 'RICO']


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

RICO stands for the Racketeer Influenced and Corrupt Organizations Act, which targets organized criminal activity and racketeering. RICO enhances existing criminal punishments and creates new causes of action for acts done as a part of an organized criminal enterprise. This can apply to a pharmaceutical kickbacks case because that also involves people forming an enterprise to violate federal and state laws to enrich themselves.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## your code here
date_pattern = r"\b(years?|YEAR|YEARS?)\b"

date_years = [tok.text for tok in pharma_ner.ents if tok.label_ == "DATE" and re.search(date_pattern, tok.text)]
print(date_years)


['last year', '20 years', 'three years', 'five years', 'three years']


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [10]:
## your code here
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', pharma)

# Find sentences that contain the words in date_years
matching_sentences = [sentence for sentence in sentences if any(word in sentence for word in date_years)]
print(matching_sentences)

['"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.', 'Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.', ' The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.']


If convicted, the CEO would face a maximum of 20 years in prison and three years of supervised release.

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [11]:
## your code here for subsetting
doj_subset = doj[doj.topics_clean.isin(["Civil Rights", "Hate Crimes", "Project Safe Childhood"])]
len(doj_subset)

717

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [12]:
## your code here to define function
sent_obj = SentimentIntensityAnalyzer()
def sent_analysis(pr_string):
    # find named entities
    doc = nlp(pr_string)
    for ent in doc.ents:
        pr = re.sub(re.escape(ent.text), "", pr_string)
    
    # score the sentiment
    sent_pr = sent_obj.polarity_scores(pr)
    
    # return sentiment dictionary
    return sent_pr

In [13]:
## your code here executing the function
doj_subset["sentiment"] = doj_subset["contents"].apply(sent_analysis)
doj_subset["sentiment"]

77        {'neg': 0.163, 'neu': 0.768, 'pos': 0.07, 'compound': -0.9871}
155       {'neg': 0.124, 'neu': 0.795, 'pos': 0.081, 'compound': -0.926}
157       {'neg': 0.09, 'neu': 0.826, 'pos': 0.084, 'compound': -0.5719}
162      {'neg': 0.118, 'neu': 0.796, 'pos': 0.086, 'compound': -0.9037}
168      {'neg': 0.148, 'neu': 0.815, 'pos': 0.037, 'compound': -0.9864}
                                      ...                               
13002      {'neg': 0.136, 'neu': 0.8, 'pos': 0.064, 'compound': -0.9645}
13032      {'neg': 0.078, 'neu': 0.82, 'pos': 0.102, 'compound': 0.8225}
13034    {'neg': 0.131, 'neu': 0.735, 'pos': 0.135, 'compound': -0.0516}
13068     {'neg': 0.125, 'neu': 0.745, 'pos': 0.13, 'compound': -0.2054}
13081     {'neg': 0.13, 'neu': 0.836, 'pos': 0.034, 'compound': -0.9898}
Name: sentiment, Length: 717, dtype: object

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [14]:
## your code here
doj_subset_wscore = doj_subset
scores = ["neg", "neu", "pos", "compound"]
for score in scores:
    doj_subset_wscore[score] = doj_subset_wscore["sentiment"].apply(lambda x: x[score])

doj_subset_wscore.sort_values(by="neg", ascending = False, inplace = True)
doj_subset_wscore[["id", "contents", "neg"]].head(2)

,id,contents,neg
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.259
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",0.255


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [15]:
## agg and find the mean compound score by topic
doj_subset_wscore.groupby("topics_clean").agg({"compound": "mean"})

,compound
topics_clean,
Civil Rights,0.159683
Hate Crimes,-0.890578
Project Safe Childhood,-0.602623


The variation in compound sentiment scores for the different topics could be because of the different natures of each topic. Hate crimes often involves hate speech and other hateful actions, and project safe childhood involves quite a lot of exploitation and abuse. On the other hand, civil rights maybe involve a greater range of cases and are oftentimes less severe than hate crimes or child protection cases.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [16]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [17]:
## your code defining a text processing function
def process(string):
    # convert to lowercase
    lower = string.lower()

    # combine stopwords
    list_stopwords = stopwords.words("english")
    all_stopwords = list_stopwords + custom_doj_stopwords
    
    # remove stopwords
    nostops = [word for word in wordpunct_tokenize(lower) if word not in all_stopwords]
    
    # snowball stemmer
    snow_stemmer = SnowballStemmer("english")
    processed = [snow_stemmer.stem(token) for token in nostops if token.isalpha() and len(token) >= 4]
    
    return " ".join(processed)
    

In [18]:
## your code executing the function
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(process)

In [19]:
## your code showing the examples
doj_subset_wscore[["id", "contents", "processed_text"]][(doj_subset_wscore.id == "16-718") 
                                                        | (doj_subset_wscore.id == "16-217")]

,id,contents,processed_text
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",nine count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [20]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out ())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [21]:
# your code here
# create document-term matrix
dtm_df = create_dtm(list_of_strings = doj_subset_wscore.processed_text,
                metadata = 
                doj_subset_wscore[["id", "compound", "topics_clean"]])
dtm_df.head()

,index,id,compound,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealous,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
0,329,14-248,-0.9943,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11593,16-718,-0.9964,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,572,13-312,-0.9980,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,501,11-626,-0.9985,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11876,15-1348,-0.9963,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# define function
def get_topwords(subset):
    top_terms = subset.drop(columns=['id', 'compound', 'topics_clean', 'index']).sum()
    top_terms = top_terms.sort_values(ascending = False).head(10)
    return top_terms
    

In [23]:
# top 5% compound sentiments
top_5_percent = dtm_df.iloc[:, 2].quantile(0.95)
top_5_df = dtm_df[dtm_df.iloc[:, 2] >= top_5_percent]
get_topwords(top_5_df)
    

agreement     161
disabl        132
enforc        122
state         107
settlement    103
ensur         101
communiti      99
court          94
hous           91
student        85
dtype: int64

In [24]:
# bottom 5% 
bottom_5_percent = dtm_df.iloc[:, 2].quantile(0.05)
bottom_5_df = dtm_df[dtm_df.iloc[:, 2] <= bottom_5_percent]
get_topwords(bottom_5_df)

assault     191
crime       167
victim      151
hate        133
conspir     120
defend      118
offic       117
american    110
african     100
charg       100
dtype: int64

In [25]:
# by topic
topics = ["Hate Crimes", "Civil Rights", "Project Safe Childhood"]
for topic in topics:
    topic_df = dtm_df[dtm_df['topics_clean'] == topic]
    print(topic, "\n", get_topwords(topic_df))

Hate Crimes 
 victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64
Civil Rights 
 offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
court        414
dtype: int64
Project Safe Childhood 
 child          1022
exploit         701
sexual          572
safe            479
childhood       474
project         472
pornographi     452
children        423
crimin          405
prosecut        374
dtype: int64


## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [26]:
lower_bound = round(len(doj_subset_wscore) * 0.05) 
upper_bound = round(len(doj_subset_wscore) * 0.95) 

civil_rights_topic = doj_subset_wscore[doj_subset_wscore['topics_clean'] == 'Civil Rights'].processed_text.tolist()
hate_crimes_topic = doj_subset_wscore[doj_subset_wscore['topics_clean'] == 'Hate Crimes'].processed_text.tolist()
safe_child_topic = doj_subset_wscore[doj_subset_wscore['topics_clean'] == 'Project Safe Childhood'].processed_text.tolist()

civil_rights_tokens = [wordpunct_tokenize(one_text) for one_text in civil_rights_topic]
hate_crimes_tokens = [wordpunct_tokenize(one_text) for one_text in hate_crimes_topic]
safe_child_tokens = [wordpunct_tokenize(one_text) for one_text in safe_child_topic]

all_tokens = civil_rights_tokens + hate_crimes_tokens + safe_child_tokens

text_raw_dict = corpora.Dictionary(all_tokens)

text_raw_dict.filter_extremes(no_below=lower_bound, no_above=upper_bound)

corpus_fromdict = [text_raw_dict.doc2bow(one_text) for one_text in all_tokens]

In [27]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_fromdict, num_topics=3, id2word=text_raw_dict, passes=6, alpha='auto', per_word_topics=True, random_state = 43)

In [28]:
topics = ldamodel.print_topics(num_words = 15)

for topic in topics:
    print(topic)

(0, '0.034*"child" + 0.023*"exploit" + 0.020*"sexual" + 0.016*"safe" + 0.016*"project" + 0.015*"childhood" + 0.015*"pornographi" + 0.014*"crimin" + 0.014*"children" + 0.013*"prosecut" + 0.012*"sentenc" + 0.012*"victim" + 0.011*"minor" + 0.010*"ceo" + 0.010*"year"')
(1, '0.019*"hous" + 0.017*"discrimin" + 0.015*"disabl" + 0.011*"said" + 0.011*"enforc" + 0.010*"agreement" + 0.010*"state" + 0.010*"court" + 0.009*"alleg" + 0.009*"fair" + 0.009*"requir" + 0.008*"settlement" + 0.008*"feder" + 0.008*"general" + 0.008*"inform"')
(2, '0.016*"victim" + 0.015*"charg" + 0.014*"sentenc" + 0.013*"prosecut" + 0.013*"defend" + 0.013*"feder" + 0.013*"crime" + 0.012*"guilti" + 0.012*"said" + 0.011*"hate" + 0.011*"indict" + 0.011*"year" + 0.010*"prison" + 0.010*"investig" + 0.010*"american"')


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [29]:
## your code here to get doc-level topic probabilities 

lda=[ldamodel.get_document_topics(word, minimum_probability=0) for word in corpus_fromdict]

assert len(lda) == len(doj_subset_wscore)

In [30]:
## your code here to add those topic probabilities to the dataframe

lda_topics = ldamodel.num_topics

for i in range(lda_topics):
    column_name = f"topic_{i}"
    topic_probs = [prob[i][1] if len(prob) > i else 0.0 for prob in lda]
    doj_subset_wscore[column_name] = topic_probs
    
def get_top_topic(row):
    topic_columns = ["topic_0", "topic_1", "topic_2"]
    probabilities = [row[column] for column in topic_columns]
    top_topic_index = probabilities.index(max(probabilities))
    return top_topic_index + 1

doj_subset_wscore['top_topic'] = doj_subset_wscore.apply(get_top_topic, axis = 1)

In [31]:
## your code here to summarize the topic proportions for each of the topics_clean 

freq_table = pd.crosstab(doj_subset_wscore['topics_clean'], doj_subset_wscore['top_topic'], normalize='index')
print(freq_table)

top_topic                      1         2         3
topics_clean                                        
Civil Rights            0.498361  0.183607  0.318033
Hate Crimes             0.008130  0.443089  0.548780
Project Safe Childhood  0.066265  0.216867  0.716867


# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [32]:
# A
def create_bigram_onedoc(processed_text):

    wordlist = [word for word in word_tokenize(str(processed_text)) 
                if word.isalpha()]

    bigram_list = []
    
    for i in range(len(wordlist) - 1):
        bigram_list.append(wordlist[i] + '_' + wordlist[i + 1])
    
    bigram_text = " ".join(bigram_list)
    
    return bigram_text

doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)

In [33]:
# B
doj_subset_wscore[['id', 'processed_text', 'processed_text_bigrams']][(doj_subset_wscore.id == '16-217')]

,id,processed_text,processed_text_bigrams
6727,16-217,reach comprehens settlement agreement citi miami miami polic resolv offic involv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quick investig offic involv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss vote settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_offic offic_involv involv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_offic offic_involv involv_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_offic offic_involv involv_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_offic offic_involv involv_shoot shoot_effect effect_quick quick_investig investig_offic offic_involv involv_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa or

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

In [34]:
# C
dtm_bigram = create_dtm(list_of_strings = doj_subset_wscore.processed_text_bigrams,
                metadata = doj_subset_wscore[["id", "compound", "topics_clean"]])

dtm_bigram.head()

,index,id,compound,topics_clean,aaron_ford,aaron_latham,aaron_mcgrath,aaron_parrish,aaron_polster,aaron_rice,...,zone_religi,zone_request,zone_restrict,zone_student,zone_time,zone_varianc,zunggeemog_noel,zunggeemog_prompt,zunggeemog_write,zwengel_princeton
0,329,14-248,-0.9943,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11593,16-718,-0.9964,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,572,13-312,-0.9980,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,501,11-626,-0.9985,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11876,15-1348,-0.9963,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# D
dtm_df.shape
dtm_bigram.shape

(717, 6870)

(717, 72723)

ANSWER D: The bigram matrix has more dimensions than than the unigram matrix because bigram magnifies the number of words by grouping the same word multiple times (with the word before it and with the word after it). 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [52]:
# your code here

topics_names = ['Hate Crimes', 'Civil Rights', 'Project Safe Childhood']

for topic in topics_names:
    topwords = get_topwords(dtm_bigram[dtm_bigram["topics_clean"] == topic])
    print(topic, '\n', topwords)

Hate Crimes 
 hate_crime          379
african_american    367
plead_guilti        275
year_prison         161
special_agent       124
racial_motiv        114
thoma_perez         111
grand_juri          101
perez_general        95
said_thoma           91
dtype: int64
Civil Rights 
 fair_hous         231
princip_deputi    221
deputi_general    221
vanita_gupta      202
gupta_head        200
general_vanita    199
said_princip      186
unit_state        156
nation_origin     143
consent_decre     128
dtype: int64
Project Safe Childhood 
 safe_childhood       474
project_safe         472
child_pornographi    450
child_exploit        281
sexual_exploit       223
exploit_children     200
plead_guilti         197
exploit_obscen       176
obscen_section       175
child_sexual         174
dtype: int64


# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [37]:
# your code here 